In [1]:
import pandas as pd

!pwd

/Users/tomrix/Library/CloudStorage/OneDrive-ImperialCollegeLondon/projects/untargeted_sspa


In [2]:
pd.read_csv('./data/trans_omic_untargeted/MTBLS2542_neg_xcms_data.tsv', delimiter='\t')

,name,namecustom,mz,mzmin,mzmax,rt,rtmin,rtmax,npeaks,.,...,20P02820306_BEHC18_NEG,20P02820307_BEHC18_NEG,20P02820310_BEHC18_NEG,20P02820315_BEHC18_NEG,20P02820317_BEHC18_NEG,20P02820320_BEHC18_NEG,20P02820322_BEHC18_NEG,20P02820323_BEHC18_NEG,20P02820326_BEHC18_NEG,20P02820328_BEHC18_NEG
0,M70T601,M70.06051T600.8048,70.060510,70.060469,70.060615,600.804840,585.897217,601.959900,56,56,...,36760.650998,59458.771796,0.000000,0.000000,0.000000,15628.965076,65379.112754,0.000000,0.000000,0.000000
1,M70T438_1,M70.139937T437.7077_1,70.139937,70.139711,70.146540,437.707748,420.291290,445.335815,140,137,...,225602.000179,186400.905250,332967.978283,147129.923839,68646.123073,346203.863608,432146.438338,178401.237376,209440.879111,411384.319876
2,M70T438_2,M70.251502T437.704_2,70.251502,70.251303,70.251866,437.703979,421.962006,438.700958,97,97,...,63303.563516,48090.787293,93230.210955,38791.369800,17009.258659,101033.646010,133930.526451,45296.639221,51312.147237,121093.031273
3,M70T584,M70.259705T583.6155,70.259705,70.259526,70.260006,583.615540,568.318298,584.827209,143,143,...,88934.362077,108019.511449,101230.975036,133869.201805,119490.965645,97649.210049,43535.729769,155855.101489,30448.954110,119810.170665
4,M70T83,M70.26043T83.225,70.260430,70.260383,70.260460,83.224968,81.378479,86.558502,93,93,...,0.000000,0.000000,13584.403731,10924.970442,6507.385018,0.000000,8879.425096,0.000000,0.000000,22773.087268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23690,M1038T594,M1037.671523T593.8992,1037.671523,1037.670188,1037.673849,593.899231,576.969910,594.721375,150,150,...,89637.148886,277291.216804,576305.398974,409168.840969,482905.153439,432920.131207,140681.136360,338178.237227,125437.758210,506517.718254
23691,M1038T544,M1037.679218T543.5695,1037.679218,1037.671987,1037.680755,543.569458,529.295044,561.751099,43,43,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
23692,M1039T594,M1038.676457T593.8976,1038.676457,1038.674153,1038.679008,593.897583,577.597107,594.531433,105,105,...,0.000000,61222.401061,146126.690647,86971.252015,105852.585070,46572.047301,0.000000,0.000000,0.000000,108952.542620
23693,M1044T582,M1043.613777T582.3431,1043.613777,1043.612279,1043.616791,582.343140,581.660278,583.036865,45,45,...,0.000000,57603.908159,88846.277470,0.000000,0.000000,0.000000,0.000000,44646.287285,0.000000,0.000000
